In [1]:
import os
import numpy as np
import torch
from tqdm import tqdm
import pickle as pkl
import time

import librosa
import torch
import torchaudio
from transformers import AutoProcessor, EncodecModel, EncodecFeatureExtractor
from utils import DanceToMusic


/Users/azeez/miniforge3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
if torch.cuda.is_available():
        device = torch.device("cuda:0")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
device = torch.device("cpu")


model_id = "facebook/encodec_24khz"
encodec_model = EncodecModel.from_pretrained(model_id)
codebook_size = encodec_model.quantizer.codebook_size
encodec_model.to(device)
sample_rate = 24000

data_dir = "/Users/azeez/Documents/pose_estimation/DanceToMusic/data/min_training_data"
train_dataset = DanceToMusic(data_dir, encoder = encodec_model, sample_rate = sample_rate, device=device)

/Users/azeez/miniforge3/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [19]:
audio_codes, pose, pose_mask, wav, wav_mask, _, _ = train_dataset[0]
# encodec_model.set_target_bandwidth(6.0)
low_audio_codes = encodec_model.encode(wav.unsqueeze(0), wav_mask)['audio_codes']
print("audio codes shape: ", audio_codes.shape)
encodec = EncodecModel.encodec_model_24khz()
print("low audio codes shape: ", low_audio_codes.shape)

audio codes shape:  torch.Size([1, 1503])


AttributeError: type object 'EncodecModel' has no attribute 'encodec_model_24khz'

In [63]:
from encodec import EncodecModel

model = EncodecModel.encodec_model_24khz()
# The number of codebooks used will be determined bythe bandwidth selected.
# E.g. for a bandwidth of 6kbps, `n_q = 8` codebooks are used.
# Supported bandwidths are 1.5kbps (n_q = 2), 3 kbps (n_q = 4), 6 kbps (n_q = 8) and 12 kbps (n_q =16) and 24kbps (n_q=32).
# For the 48 kHz model, only 3, 6, 12, and 24 kbps are supported. The number
# of codebooks for each is half that of the 24 kHz model as the frame rate is twice as much.
model.set_target_bandwidth(3)
# out = model.encode(wav.unsqueeze(0))
# out[0].shape

# Extract discrete codes from EnCodec
with torch.no_grad():
    encoded_frames = model.encode(wav.unsqueeze(0))
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)
codes.shape

/Users/azeez/miniforge3/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


torch.Size([1, 4, 751])

In [76]:
from IPython.display import Audio, display

w = model.decode(encoded_frames)
print(encoded_frames[0][0].shape)
display(Audio(w[0].detach().numpy(), rate=24000))

torch.Size([1, 4, 751])


In [81]:
# Convert the tuple to a list
encoded_frames_list = list(encoded_frames)

# Create a new tensor with the desired modification at [0][0]
original_tensor = encoded_frames[0][0]  # Assuming this is shape [1, 4, 751]
new_tensor = torch.zeros_like(original_tensor)
new_tensor[:, 0, :] = original_tensor.sum(dim=1)

# Replace the original tensor at [0][0] with the new tensor
encoded_frames_list[0] = list(encoded_frames_list[0])
encoded_frames_list[0][0] = new_tensor
encoded_frames_list[0] = tuple(encoded_frames_list[0])

# Convert the list back to a tuple
modified_encoded_frames = tuple(encoded_frames_list)
one_w = model.decode(modified_encoded_frames)
display(Audio(one_w[0].detach().numpy(), rate=24000))

tensor([[[3286, 2922, 2394,  ..., 2745, 2163, 2840],
         [   0,    0,    0,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0]]])


IndexError: index out of range in self